In [1]:
import pandas as pd
import numpy as np

In [2]:
import mlflow
# mlflow.set_tracking_uri("https://mlflow.cs.ait.ac.th")
mlflow.set_experiment(experiment_name="st121413_a3")

<Experiment: artifact_location='mlflow-artifacts:/242179200426103649', creation_time=1695230678278, experiment_id='242179200426103649', last_update_time=1695230678278, lifecycle_stage='active', name='st121413_a3', tags={}>

In [3]:
x1 = np.arange(0,10)
x2 = np.arange(0,10)
X = np.array(np.meshgrid(x1,x2)).T.reshape(-1,2)

In [4]:
data = pd.DataFrame(data=X, columns=['x1','x2'])
data

,x1,x2
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
95,9,5
96,9,6
97,9,7
98,9,8


In [5]:
y = data.apply(lambda X: X.x1+X.x2, axis=1)
data['y'] = y
data.set_index(['x1','x2'], inplace=True)
data

y
x1 x2    
0  0    0
   1    1
   2    2
   3    3
   4    4
...    ..
9  5   14
   6   15
   7   16
   8   17
   9   18

[100 rows x 1 columns]

In [6]:
data.to_csv('./dataset/dataset.csv')

Train model

In [7]:
# Load Dataset
data = pd.read_csv('./dataset/dataset.csv')
data

,x1,x2,y
0,0,0,0
1,0,1,1
2,0,2,2
3,0,3,3
4,0,4,4
...,...,...,...
95,9,5,14
96,9,6,15
97,9,7,16
98,9,8,17


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

train, test = train_test_split(data, test_size=0.2)
with mlflow.start_run():
    print(f"{train.shape=} {test.shape=}")
    train = mlflow.data.from_pandas(train)
    mlflow.log_input(train, context="train")
    test = mlflow.data.from_pandas(test)
    mlflow.log_input(test, context="test")

    X = train.df[['x1','x2']]
    y = train.df[['y']]
    model = LinearRegression()
    model.fit(X,y)
    print(f"{model.coef_=}")

    mlflow.sklearn.log_model(model, artifact_path="model")

train.shape=(80, 3) test.shape=(20, 3)


/usr/local/lib/python3.11/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
/usr/local/lib/python3.11/site-packages/mlflow/data/pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  return _infer_schema(self._d

model.coef_=array([[1., 1.]])


In [35]:
from sklearn.metrics import mean_squared_error
# Test the model
pred = model.predict(test[['x1','x2']])
mse = mean_squared_error(y_pred=pred, y_true=test[['y']])
print(f"{mse=}")

mse=3.83583615163717e-30


Save and Load model

In [1]:
from utils import *
import mlflow

In [2]:
model = load_mlflow(stage='Staging')
print(f"{model.coef_=}")

model.coef_=array([[1., 1.]])


In [8]:
import pandas as pd
import numpy as np

In [9]:
x_1 = 1
x_2 = 10
X = np.array([x_1,x_2]).reshape(-1,2)
X = pd.DataFrame(X, columns=['x1', 'x2']) 
X

,x1,x2
0,1,10


In [10]:
a = model.predict(X)

In [11]:
a[0][0]

11.000000000000002

In [9]:
from mlflow.client import MlflowClient
client = MlflowClient()

In [28]:
r = client.get_registered_model("st121413-a3")

In [30]:
r

<RegisteredModel: aliases={}, creation_timestamp=1695231138279, description='', last_updated_timestamp=1695243419179, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1695231156369, current_stage='Production', description='', last_updated_timestamp=1695243366699, name='st121413-a3', run_id='0da61b8046e34223a4c7baac6e9dd7d3', run_link='', source='mlflow-artifacts:/242179200426103649/0da61b8046e34223a4c7baac6e9dd7d3/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>,
 <ModelVersion: aliases=[], creation_timestamp=1695243408942, current_stage='Staging', description='', last_updated_timestamp=1695243419179, name='st121413-a3', run_id='37bf1137e2b743599f8d70fafe431953', run_link='', source='mlflow-artifacts:/242179200426103649/37bf1137e2b743599f8d70fafe431953/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>], name='st121413-a3', tags={}>

In [29]:
for m in r:
    print(m)

('aliases', {})
('creation_timestamp', 1695231138279)
('description', '')
('last_updated_timestamp', 1695243419179)
('latest_versions', [<ModelVersion: aliases=[], creation_timestamp=1695231156369, current_stage='Production', description='', last_updated_timestamp=1695243366699, name='st121413-a3', run_id='0da61b8046e34223a4c7baac6e9dd7d3', run_link='', source='mlflow-artifacts:/242179200426103649/0da61b8046e34223a4c7baac6e9dd7d3/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>, <ModelVersion: aliases=[], creation_timestamp=1695243408942, current_stage='Staging', description='', last_updated_timestamp=1695243419179, name='st121413-a3', run_id='37bf1137e2b743599f8d70fafe431953', run_link='', source='mlflow-artifacts:/242179200426103649/37bf1137e2b743599f8d70fafe431953/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>])
('name', 'st121413-a3')
('tags', {})


In [23]:
m = r.latest_versions[0]

In [27]:
m.version, m.current_stage

('1', 'Production')